# ATRIUM Task 4.1.3 Experiment

This notebook demonstrates applying the USW vocabulary-based NER pipeline to records in a JSONL data file supplied by ATHENA, and merging the output results for further subsequent processing.
The pipeline identifies terms originating from the following three Linked Open Data controlled vocabularies:

* [FISH Event Types Thesaurus](http://purl.org/heritagedata/schemes/agl_et)
* [FISH Archaeological Sciences Thesaurus](http://purl.org/heritagedata/schemes/560)
* [Getty Art & Architecture Thesaurus - Activities facet](https://vocab.getty.edu/aat/300404112)

The [Forum on Information Standards in Heritage (FISH)](https://heritage-standards.org.uk/) terminology working group administer and maintain UK national standard controlled terminologies for cultural heritage - known as the [FISH Vocabularies](https://heritage-standards.org.uk/fish-vocabularies/). These are made freely available for download and use as Linked Open Data via the [Heritage Data](https://www.heritagedata.org/) site. 

The [Getty Art & Architecture Thesaurus (AAT)](http://vocab.getty.edu/aat/) is made available online as Linked Open Data. The (poly)hierarchical structure is subdivided at the top level into a number of facets. We have isolated terms originating from the [AAT Activities facet](https://vocab.getty.edu/aat/300404112) for this exercise.

The listing following the source code sections below shows actual output of the process. These results are also merged within the originating input data under the 'spans' section and saved as a new file. The following example illustrates how the spans identified in the 3rd result record are merged into the existing 'spans' section:

![alt text](img/added-span-results.png "Added span results")



In [1]:
%%capture
import warnings
# suppress user warnings during execution
warnings.filterwarnings(action='ignore', category=UserWarning)
warnings.filterwarnings(action='ignore', category=FutureWarning)

# install prerequisites
%pip install spacy
%pip install srsly
%sx python -m spacy download en_core_web_sm

In [2]:
import spacy # for NER processing
from spacy import displacy # for visualisation of NER tagged text
import srsly # for JSONL serialization/deserialization
import os
from slugify import slugify # for valid filenames
from rematch2 import VocabularyRuler, DocSummary, StringCleaning, child_span_remover # custom vocabulary-based NER components
from IPython.display import display, HTML


# check if a given span exists in a list of spans
# comparing start position, end position and label
def span_exists(span: dict, lst: list) -> bool:
    start = span.get("start", "")
    end = span.get("end", "")
    label = span.get("label", "")
    return any(
        item.get("start") == start 
        and item.get("end") == end 
        and item.get("label") == label for item in lst
    )           


if __name__ == '__main__':
    # read JSONL input data from file
    input_file_path = "./data/athena"
    input_file_name1 = "sample_input.jsonl"
    input_file_name2 = "sample_annotated_output.jsonl"
    #input_file_name3 = "journal_metadata.jsonl"
    #input_file_name4 = "report_metadata.jsonl"

    # change this to process other files   
    input_file_name = input_file_name2 

    input_file_full = os.path.join(input_file_path, input_file_name) 
    data = list(srsly.read_jsonl(input_file_full))

    # set up default base NER pipeline (English)
    nlp = spacy.load("en_core_web_sm", disable = ['ner'])
    
    # add custom pipeline NER components
    nlp.add_pipe("fish_event_types_ruler", last=True)   
    nlp.add_pipe("fish_archsciences_ruler", last=True)   
    nlp.add_pipe("aat_activities_ruler", last=True)  
    nlp.add_pipe("child_span_remover", last=True) 
    
    # process each item in the input data
    for item in data:
        identifier = item.get("meta", {}).get("id", "").strip()
        text = StringCleaning.normalize_text(item.get("text", ""))
        
        # run NER pipeline against input text
        doc = nlp(text)
        
        # display HTML summary of NER results (see below)
        summary = DocSummary(doc)
        display(HTML(f"<h3>[ID: {identifier}]</h3>"))
        if(len(summary.spans(format="list")) == 0):
            display(text)
        else:
            display(HTML(summary.doctext(format="html")))
        
        display(HTML(summary.spans(format="html")))
        #display(HTML(summary.tokens(format="html")))
        display(HTML("<hr>"))

        # add new spans to the existing spans array,
        # checking for duplicates (in case multiple runs)
        the_spans = item.get("spans", []) 
        new_spans = summary.spans(format="list")
        for span in new_spans:
            if not span_exists(span, the_spans):
                the_spans.append(span)
        item["spans"] = the_spans
    
    # create output file path if it does not already exist
    output_file_path = os.path.join(input_file_path, "output")
    if not os.path.exists(output_file_path):
        os.makedirs(output_file_path)

    # output the modified structure to a (new) JSONL file    
    output_file_name = os.path.join(output_file_path, f"{slugify(input_file_name)}-plus-vocab-ner.jsonl") 
    srsly.write_jsonl(output_file_name, data) 


start,end,token_start,token_end,label,id,text
22,38,2,3,AAT_ACTIVITY,http://vocab.getty.edu/aat/300223990,spatial analysis
22,38,2,3,AAT_ACTIVITY,http://vocab.getty.edu/aat/300054595,spatial analysis


start,end,token_start,token_end,label,id,text
69,79,9,9,FISH_ARCHSCIENCE,http://purl.org/heritagedata/schemes/560/concepts/142118,carbonised
13,22,2,2,AAT_ACTIVITY,http://vocab.getty.edu/aat/300077121,collected
69,79,9,9,AAT_ACTIVITY,http://vocab.getty.edu/aat/300379618,carbonised
173,182,41,41,AAT_ACTIVITY,http://vocab.getty.edu/aat/300138076,processed
173,182,41,41,AAT_ACTIVITY,http://vocab.getty.edu/aat/300078065,processed


start,end,token_start,token_end,label,id,text
41,64,7,8,FISH_ARCHSCIENCE,http://purl.org/heritagedata/schemes/560/concepts/142158,magnetic susceptibility
101,108,14,14,AAT_ACTIVITY,http://vocab.getty.edu/aat/300137584,provide


start,end,token_start,token_end,label,id,text
76,93,15,16,FISH_ARCHSCIENCE,http://purl.org/heritagedata/schemes/560/concepts/142188,radiocarbon dated
76,93,15,16,AAT_ACTIVITY,http://vocab.getty.edu/aat/300054717,radiocarbon dated
218,222,37,37,AAT_ACTIVITY,http://vocab.getty.edu/aat/300404795,date
218,222,37,37,AAT_ACTIVITY,http://vocab.getty.edu/aat/300054714,date


start,end,token_start,token_end,label,id,text
0,10,0,0,AAT_ACTIVITY,http://vocab.getty.edu/aat/300380461,Calibrated
190,217,31,33,AAT_ACTIVITY,http://vocab.getty.edu/aat/300081742,neutron activation analysis


...

start,end,token_start,token_end,label,id,text
16,24,3,3,AAT_ACTIVITY,http://vocab.getty.edu/aat/300226216,examined


start,end,token_start,token_end,label,id,text
14,20,2,2,AAT_ACTIVITY,http://vocab.getty.edu/aat/300053130,models
14,20,2,2,AAT_ACTIVITY,http://vocab.getty.edu/aat/300155050,models
138,143,31,31,AAT_ACTIVITY,http://vocab.getty.edu/aat/300053097,fixed


start,end,token_start,token_end,label,id,text
14,40,2,3,AAT_ACTIVITY,http://vocab.getty.edu/aat/300054595,phytosociological analysis
127,135,17,17,AAT_ACTIVITY,http://vocab.getty.edu/aat/300236367,adjusted


start,end,token_start,token_end,label,id,text
13,35,3,5,AAT_ACTIVITY,http://vocab.getty.edu/aat/300081693,trace element analysis
218,229,38,38,AAT_ACTIVITY,http://vocab.getty.edu/aat/300053892,transported


start,end,token_start,token_end,label,id,text
0,7,0,0,AAT_ACTIVITY,http://vocab.getty.edu/aat/300239496,Running
30,38,5,5,AAT_ACTIVITY,http://vocab.getty.edu/aat/300053130,modelled
30,38,5,5,AAT_ACTIVITY,http://vocab.getty.edu/aat/300155050,modelled
49,57,8,8,AAT_ACTIVITY,http://vocab.getty.edu/aat/300054595,analyzed
